<center> <h1> Restoration of underwater images </h1> <br> <h3> <br> 4MEO Computer vision 2</h3> </center>

# I. Introduction

### 1. Import libraries

In [84]:
import numpy as np
import scipy as sc
import cv2 as cv
import math
import matplotlib.image as mpimg
import matplotlib.pylab as plt

# II. Compute inputs

## 1. Show function

In [85]:
def show(final):
    cv.imshow('White balanced', final)
    cv.waitKey(0)
    cv.destroyAllWindows()
    for i in range (1,5):
        cv.waitKey(1)

## 2. White balance function definition 

In [86]:
def white_balance(img):
    result = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv.cvtColor(result, cv.COLOR_LAB2BGR)
    return result

## 3. Enhance contrast function definition 

In [87]:
def enhance_contrast(im):
    rgb_img = im

    # convert from RGB color-space to YCrCb
    ycrcb_img = cv.cvtColor(rgb_img, cv.COLOR_BGR2YCrCb)

    # equalize the histogram of the Y channel
    ycrcb_img[:, :, 0] = cv.equalizeHist(ycrcb_img[:, :, 0])

    # convert back to RGB color-space from YCrCb
    equalized_img = cv.cvtColor(ycrcb_img, cv.COLOR_YCrCb2BGR)
    return equalized_img

## 4. Luminance weight map

In [88]:
def luminance_weight_map(img):
    im = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    L = np.mean(im,axis=2) #mean for each axes
    lumWeightMap = np.sqrt((1/3)*np.square((im[:,:,0]-L))+np.square((im[:,:,1]-L))+np.square((im[:,:,2]-L)))
    return lumWeightMap

## 5. Chromatic weight map

In [89]:
def chromatic_weight_map(img):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    saturationValue = hsv[:,:,1]; 
    saturationMax = 1
    sigma = 0.3
    chromaticWeightmap = np.exp( -1 * ((np.square(saturationValue - saturationMax)) / (2*np.square(sigma))) )
    return chromaticWeightmap

## 6. Saliency weight map

In [91]:
def saliency_weight_map(img):
    saliency = cv.saliency.StaticSaliencySpectralResidual_create()
    (success, saliencyMap) = saliency.computeSaliency(img)
    saliencyMap = (saliencyMap * 255).astype("uint8")
    return saliencyMap

## Output

In [99]:
img = cv.imread('data/test4.png')

firstInput = white_balance(img)
secondInput = enhance_contrast(firstInput)
#final = np.hstack((img, firstInput, secondInput))
#how(final)

luminanceWeightmap1 = luminance_weight_map(firstInput)
chromaticWeightmap1 = chromatic_weight_map(firstInput)
saliencyWeightmap1 = saliency_weight_map(firstInput)
resultedWeightmap1 = np.multiply(np.multiply(luminanceWeightmap1,chromaticWeightmap1),saliencyWeightmap1)

luminanceWeightmap2 = luminance_weight_map(secondInput)
chromaticWeightmap2 = chromatic_weight_map(secondInput)
saliencyWeightmap2 = saliency_weight_map(secondInput)
resultedWeightmap2 = np.multiply(np.multiply(luminanceWeightmap2,chromaticWeightmap2),saliencyWeightmap2)

normaizedWeightmap1 = np.divide(resultedWeightmap1,(resultedWeightmap1 + resultedWeightmap2))
normaizedWeightmap2 = np.divide(resultedWeightmap2,(resultedWeightmap1 + resultedWeightmap2))

print(normaizedWeightmap1)

[[1.00000000e+000 1.00000000e+000 1.00000000e+000 ... 1.00000000e+000
              nan 0.00000000e+000]
 [1.00000000e+000 1.00000000e+000 1.00000000e+000 ... 1.00000000e+000
              nan 0.00000000e+000]
 [1.00000000e+000             nan 1.00000000e+000 ...             nan
              nan 0.00000000e+000]
 ...
 [0.00000000e+000 0.00000000e+000             nan ... 1.00000000e+000
  2.12214114e-278 1.00000000e+000]
 [0.00000000e+000 0.00000000e+000 4.56593098e-127 ... 5.24738480e-165
  0.00000000e+000 0.00000000e+000]
 [1.00000000e+000             nan 1.00000000e+000 ...             nan
              nan             nan]]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
